In [ ]:
!pip install transformers torch --upgrade

Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (2.3.0)
Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.3.1)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os

os.chdir('/content/drive/My Drive/workspace/kaggle-workspace/')

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
from tqdm import tqdm_notebook
import transformers as ppb
import torch

def get_word_vecs(docs):
    # For DistilBERT:
    model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)
    DEVICE = torch.device("cuda")
    model.to(DEVICE)

    def chunks(l, n):
        #Yield successive n-sized chunks from l.
        for i in range(0, len(l), n):
            yield l[i:i + n]

    batch_size = 10

    fin_features = []
    for data in tqdm_notebook(chunks(docs, batch_size)):
        tokenized = []
        for x in data:
            x = " ".join(x.strip().split()[:200])
            tok = tokenizer.encode(x, add_special_tokens=True)
            tokenized.append(tok[:512])
        
        max_len = 512
        padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized])
        attention_mask = np.where(padded != 0, 1, 0)
        input_ids = torch.tensor(padded).to(DEVICE)
        attention_mask = torch.tensor(attention_mask).to(DEVICE)

        with torch.no_grad():
          last_hidden_states = model(input_ids, attention_mask=attention_mask)
      
        features = last_hidden_states[0][:, 0, :].cpu().numpy()
        fin_features.append(features)
        
    return np.vstack(fin_features)

def conv_file_to_vecs(i, o):
    df = pd.read_csv(i)
    docs = df.clean.tolist()
    vecs = get_word_vecs(docs)
    pd.concat([df.loc[:, ['id']], pd.DataFrame(vecs, columns=['vecs_%s' % i for i in range(vecs.shape[1])])], axis=1).to_csv(o, index=None)

conv_file_to_vecs('./fact/train_clean.csv', './fact/train_vecs.csv')
conv_file_to_vecs('./fact/test_clean.csv', './fact/test_vecs.csv')

In [1]:
import time


while True:
    time.sleep(3600)
    !date

SyntaxError: ignored